# Jobs
> This script fetches, processes and analyzes [job postings by Netflix](https://jobs.netflix.com/search?page=2&location=Los%20Angeles%2C%20California~Remote%2C) for its Los Angeles office. 

#### Load Python tools and Jupyter config

In [81]:
import re
import json
import sqlite3
import requests
import numpy as np
import pandas as pd
import jupyter_black
import altair as alt
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from collections import Counter

In [22]:
jupyter_black.load()
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = None

In [3]:
today = pd.Timestamp("today").strftime("%Y%m%d")

---

## Fetch

#### Headers and parameters for initial request

In [4]:
headers = {
    "authority": "jobs.netflix.com",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
}

params = {}

#### Initial request to get the total number of pages

In [5]:
response = requests.get(
    "https://jobs.netflix.com/api/search", params=params, headers=headers
)
data = response.json()
total_pages = data["info"]["postings"]["num_pages"]

#### Loop through each page to fetch postings

In [ ]:
all_postings = []

for page in tqdm(range(1, total_pages + 1)):
    params["page"] = page
    response = requests.get(
        "https://jobs.netflix.com/api/search", params=params, headers=headers
    )
    postings = response.json()["records"]["postings"]

    all_postings.extend(postings)

  0%|          | 0/20 [00:00<?, ?it/s]

---

## Process

#### Convert the list of postings into a pandas DataFrame

In [7]:
cols = [
    "external_id",
    "slug",
    "text",
    "department",
    "team",
    "state",
    "updated_at",
    "created_at",
    "location",
    "organization",
    "subteam",
    "lever_team",
    "description",
    "search_text",
]

src = pd.DataFrame(all_postings)[cols]

In [8]:
src = pd.DataFrame(all_postings)[cols]

#### Deal with lists in some columns

In [9]:
def extract_first_item(list_like):
    """Extract the first item from a list-like object or return None if empty."""
    if list_like and isinstance(list_like, list):
        return list_like[0]
    return None

#### Apply the function to each list column

In [10]:
src["team"] = src["team"].apply(extract_first_item)
src["subteam"] = src["subteam"].apply(extract_first_item)
src["organization"] = src["organization"].apply(extract_first_item)

#### Strip markup from description columns

In [11]:
def strip_html_tags(content):
    """Remove HTML tags from the provided content."""
    if content:
        soup = BeautifulSoup(content, "html.parser")
        return soup.get_text(separator=" ", strip=True)
    return content

#### Apply the function to strip HTML from the 'description' and 'search_text' columns

In [12]:
src["description"] = src["description"].apply(strip_html_tags)
src["search_text"] = src["search_text"].apply(strip_html_tags)

#### Deal with all dates

In [ ]:
src["created_at"] = pd.to_datetime(src["created_at"])
src["updated_at"] = pd.to_datetime(src["updated_at"])

---

## Extract

#### Function to extract salary ranges

In [77]:
def extract_salary(text):
    pattern = r"the\s+range\s+for\s+this\s+role\s+is\s+(\$\d{1,3}(?:,\d{3})+)(?:\s*-\s*(\$\d{1,3}(?:,\d{3})+))?"
    match = re.search(pattern, text, re.IGNORECASE)

    if match:
        return match.group(1), match.group(2) if match.group(2) else np.nan
    return np.nan, np.nan

#### Apply the function and directly split the results into two new columns

In [78]:
src["salary_lower"], src["salary_upper"] = zip(
    *src["description"].apply(extract_salary)
)

#### Function to clean and convert salary values

In [79]:
def clean_salary(salary):
    if pd.isna(salary):
        return np.nan
    return float(salary.replace("$", "").replace(",", ""))

#### Clean and convert salary columns

In [80]:
src["salary_lower"] = src["salary_lower"].apply(clean_salary)
src["salary_upper"] = src["salary_upper"].apply(clean_salary)

#### On the hunt for data science-y roles

In [117]:
keywords = {
    "data science": [
        "database",
        "data science",
        "machine learning",
        "deep learning",
        "statistics",
        "predictive modeling",
        "data analysis",
        "natural language processing",
        "NLP",
        "computer vision",
        "AI",
        "artificial intelligence",
        "python",
    ],
    "data engineering": [
        "data engineering",
        "ETL",
        "data pipeline",
        "data storage",
        "big data",
        "Hadoop",
        "Spark",
        "Apache Airflow",
        "streaming data",
        "data architecture",
        "database design",
        "SQL",
    ],
    "data visualization": [
        "visualization",
        "Tableau",
        "Power BI",
        "D3.js",
        "matplotlib",
        "plotly",
        "ggplot",
        "dashboards",
        "visual analytics",
        "data presentation",
    ],
}

#### Function to extract our keywords

In [118]:
def extract_keywords(description, keywords):
    found_keywords = []
    description_lower = description.lower()
    for category, keys in keywords.items():
        for key in keys:
            if key.lower() in description_lower and key.lower() not in found_keywords:
                found_keywords.append(key.lower())
    return found_keywords

#### Place the keywords in a column

In [119]:
src["keywords"] = src["description"].apply(lambda x: extract_keywords(x, keywords))

---

## Store

#### A clean dataframe

In [15]:
df = src.copy()

#### Connect to a SQLite database

In [16]:
conn = sqlite3.connect("data/db/netflix_jobs.db")

#### Prepare data types for the database

In [17]:
df = df.astype(str)
df["created_at"] = pd.to_datetime(df["created_at"]).astype(str)
df["updated_at"] = pd.to_datetime(df["updated_at"]).astype(str)

#### Store the dataframe as a table in the database

In [18]:
df.to_sql("jobs", conn, if_exists="replace", index=False)

390

#### Use Pandas to execute a query for 'data' jobs that are remote or in LA

In [19]:
query = """
SELECT * FROM jobs 
WHERE 
    (location LIKE '%Los Angeles%' OR location LIKE '%Remote%') 
    AND 
    team LIKE '%Data%';
"""
df_query_results = pd.read_sql_query(query, conn)

#### Display some tidy results

In [20]:
df_query_results[
    [
        "external_id",
        "slug",
        "text",
        "department",
        "team",
        "state",
        "updated_at",
        "created_at",
        "location",
        "organization",
        "subteam",
        "lever_team",
    ]
].head()

,external_id,slug,text,department,team,state,updated_at,created_at,location,organization,subteam,lever_team
0,321802958,machine-learning-scientist-l4-content-dse,Machine Learning Scientist (L4) - Content DSE,Streaming,Data Science and Engineering,Public,2024-04-03 18:30:12+00:00,2024-03-14 22:09:27+00:00,"Los Angeles, California",None,Data Science and Engineering,Content and Marketing DSE
1,321813901,capacity-planner-l5-live,"Capacity Planner, L5",Streaming,Data Platform,Public,2024-04-03 18:30:23+00:00,2024-03-12 14:34:37+00:00,"Remote, United States",None,Streaming Delivery,Streaming Delivery
2,321386898,senior-data-scientist-l5-games,"Senior Data Scientist (L5), Games",Streaming,Data Science and Engineering,Public,2024-04-03 18:30:23+00:00,2024-03-12 00:01:28+00:00,"Remote, United States",None,Data Science and Engineering,Data Science and Engineering
3,324499792,data-engineer-l4-privacy,Data Engineer (L4) - Privacy,Streaming,Data Science and Engineering,Public,2024-04-03 18:30:23+00:00,2024-02-28 20:05:07+00:00,"Remote, United States",None,Data Science and Engineering,Data Science and Engineering
4,324499162,data-engineer-l5-privacy,Data Engineer (L5) - Privacy,Streaming,Data Science and Engineering,Public,2024-04-03 18:30:16+00:00,2024-02-28 20:02:53+00:00,"Remote, United States",Products & Technology,Data Science and Engineering,Data Science and Engineering


---

## Exports

#### CSV format

In [ ]:
df.to_csv("data/processed/netflix_listings.csv", index=False)

#### JSON format

In [ ]:
df.to_json("data/processed/netflix_listings.json", indent=4, orient="records")